# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 22 2023 3:38PM,263436,10,SOTAH0001069,"Nextsource Biotechnology, LLC",Released
1,May 22 2023 3:38PM,263436,10,SOTAH0001070,"Nextsource Biotechnology, LLC",Released
2,May 22 2023 3:36PM,263435,10,Enova-11859,Emerginnova,Released
3,May 22 2023 3:15PM,263434,20,REL1-6182,Reliable 1 Laboratories LLC,Released
4,May 22 2023 3:15PM,263434,20,REL1-6183,Reliable 1 Laboratories LLC,Released
5,May 22 2023 3:15PM,263434,20,REL1-6184,Reliable 1 Laboratories LLC,Executing
6,May 22 2023 3:15PM,263434,20,REL1-6185,Reliable 1 Laboratories LLC,Released
7,May 22 2023 3:15PM,263434,20,REL1-6186,Reliable 1 Laboratories LLC,Released
8,May 22 2023 3:15PM,263434,20,REL1-6187,Reliable 1 Laboratories LLC,Released
9,May 22 2023 3:15PM,263434,20,REL1-6188,Reliable 1 Laboratories LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,263433,Released,1
33,263434,Executing,3
34,263434,Released,17
35,263435,Released,1
36,263436,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
263431,NaN,NaN,1.0
263433,NaN,NaN,1.0
263434,NaN,3.0,17.0
263435,NaN,NaN,1.0
263436,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263367,0.0,67.0,0.0
263369,0.0,1.0,0.0
263374,47.0,35.0,2.0
263376,0.0,1.0,0.0
263377,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263367,0,67,0
263369,0,1,0
263374,47,35,2
263376,0,1,0
263377,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263367,0,67,0
1,263369,0,1,0
2,263374,47,35,2
3,263376,0,1,0
4,263377,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263367,,67,
1,263369,,1,
2,263374,47,35,2
3,263376,,1,
4,263377,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 22 2023 3:38PM,263436,10,"Nextsource Biotechnology, LLC"
2,May 22 2023 3:36PM,263435,10,Emerginnova
3,May 22 2023 3:15PM,263434,20,Reliable 1 Laboratories LLC
23,May 22 2023 3:11PM,263433,20,"Exact-Rx, Inc."
24,May 22 2023 2:39PM,263431,10,"Snap Medical Industries, LLC"
25,May 22 2023 2:34PM,263426,18,"Emersa Waterbox, LLC"
26,May 22 2023 2:34PM,263427,10,Emerginnova
27,May 22 2023 2:33PM,263425,10,"Methapharm, Inc."
30,May 22 2023 2:29PM,263424,10,Methapharm-G
31,May 22 2023 2:24PM,263423,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 22 2023 3:38PM,263436,10,"Nextsource Biotechnology, LLC",,,2
1,May 22 2023 3:36PM,263435,10,Emerginnova,,,1
2,May 22 2023 3:15PM,263434,20,Reliable 1 Laboratories LLC,,3,17
3,May 22 2023 3:11PM,263433,20,"Exact-Rx, Inc.",,,1
4,May 22 2023 2:39PM,263431,10,"Snap Medical Industries, LLC",,,1
5,May 22 2023 2:34PM,263426,18,"Emersa Waterbox, LLC",,,1
6,May 22 2023 2:34PM,263427,10,Emerginnova,,,1
7,May 22 2023 2:33PM,263425,10,"Methapharm, Inc.",,,3
8,May 22 2023 2:29PM,263424,10,Methapharm-G,,,1
9,May 22 2023 2:24PM,263423,19,"AdvaGen Pharma, Ltd",,9,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 3:38PM,263436,10,"Nextsource Biotechnology, LLC",2,,
1,May 22 2023 3:36PM,263435,10,Emerginnova,1,,
2,May 22 2023 3:15PM,263434,20,Reliable 1 Laboratories LLC,17,3,
3,May 22 2023 3:11PM,263433,20,"Exact-Rx, Inc.",1,,
4,May 22 2023 2:39PM,263431,10,"Snap Medical Industries, LLC",1,,
5,May 22 2023 2:34PM,263426,18,"Emersa Waterbox, LLC",1,,
6,May 22 2023 2:34PM,263427,10,Emerginnova,1,,
7,May 22 2023 2:33PM,263425,10,"Methapharm, Inc.",3,,
8,May 22 2023 2:29PM,263424,10,Methapharm-G,1,,
9,May 22 2023 2:24PM,263423,19,"AdvaGen Pharma, Ltd",1,9,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 3:38PM,263436,10,"Nextsource Biotechnology, LLC",2,,
1,May 22 2023 3:36PM,263435,10,Emerginnova,1,,
2,May 22 2023 3:15PM,263434,20,Reliable 1 Laboratories LLC,17,3,
3,May 22 2023 3:11PM,263433,20,"Exact-Rx, Inc.",1,,
4,May 22 2023 2:39PM,263431,10,"Snap Medical Industries, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 3:38PM,263436,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
1,May 22 2023 3:36PM,263435,10,Emerginnova,1.0,NaN,NaN
2,May 22 2023 3:15PM,263434,20,Reliable 1 Laboratories LLC,17.0,3.0,NaN
3,May 22 2023 3:11PM,263433,20,"Exact-Rx, Inc.",1.0,NaN,NaN
4,May 22 2023 2:39PM,263431,10,"Snap Medical Industries, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 3:38PM,263436,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
1,May 22 2023 3:36PM,263435,10,Emerginnova,1.0,0.0,0.0
2,May 22 2023 3:15PM,263434,20,Reliable 1 Laboratories LLC,17.0,3.0,0.0
3,May 22 2023 3:11PM,263433,20,"Exact-Rx, Inc.",1.0,0.0,0.0
4,May 22 2023 2:39PM,263431,10,"Snap Medical Industries, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3951197,34.0,68.0,0.0
15,1580330,66.0,35.0,47.0
18,263426,1.0,0.0,0.0
19,1053611,7.0,32.0,8.0
20,526867,18.0,3.0,0.0
21,263376,0.0,1.0,0.0
22,263369,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3951197,34.0,68.0,0.0
1,15,1580330,66.0,35.0,47.0
2,18,263426,1.0,0.0,0.0
3,19,1053611,7.0,32.0,8.0
4,20,526867,18.0,3.0,0.0
5,21,263376,0.0,1.0,0.0
6,22,263369,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,34.0,68.0,0.0
1,15,66.0,35.0,47.0
2,18,1.0,0.0,0.0
3,19,7.0,32.0,8.0
4,20,18.0,3.0,0.0
5,21,0.0,1.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,34.0
1,15,Released,66.0
2,18,Released,1.0
3,19,Released,7.0
4,20,Released,18.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20,21,22
Status,,,,,,,
Completed,0.0,47.0,0.0,8.0,0.0,0.0,0.0
Executing,68.0,35.0,0.0,32.0,3.0,1.0,1.0
Released,34.0,66.0,1.0,7.0,18.0,0.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20,21,22
0,Completed,0.0,47.0,0.0,8.0,0.0,0.0,0.0
1,Executing,68.0,35.0,0.0,32.0,3.0,1.0,1.0
2,Released,34.0,66.0,1.0,7.0,18.0,0.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20,21,22
0,Completed,0.0,47.0,0.0,8.0,0.0,0.0,0.0
1,Executing,68.0,35.0,0.0,32.0,3.0,1.0,1.0
2,Released,34.0,66.0,1.0,7.0,18.0,0.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()